# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [53]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import string
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Christos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Christos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Christos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
# load data from database
engine = create_engine('sqlite:///disaster_messages.db')
df = pd.read_sql('DisasterMessages', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
df.drop(df[df["related"] == 2].index, inplace=True)

In [37]:
X = df.message
Y = df.iloc[:, 4:]

In [9]:
for m in X:
    if 'http' in m:
        print(m)

If you want to find a Job at an NGO or the Government, upload your resume at http://www.jobpaw.com/ 
NOTES: WHAT A JERK ,ALL HAITIANS DONT HAVE ANYTHING TO EAT ,AND ''HE'' ORDERING 3 DAYS WITHOUT FOOD LIKE SUPPORT FOR THOSE WITHOUT FOOD? http://welcome.topuertorico.org/government.shtml
http://wap.sina.comhttp://wap.sina.com 
Nokia.com http://ea.mobile.nokia.com/ea/graphics 
BEGIN:VBKM VERSION:1.0 TITLE:Digicel Live Ha URL:http://172.16.3.136/mymain2.php BEGIN:ENV X-IRMC-URLQUOTED-PRINTABLE: InternetShortcut  URLhttp://172.16.3.136/mymain2.php END:ENV END:VBKM  
BEGIN:VBKM VERSION:1.0 TITLE:Item3 URL:http://172.16.3.136/wap/news-french.php#item3 BEGIN:ENV X-IRMC-URLQUOTED-PRINTABLE: InternetShortcut  URLhttp://172.16.3.136/wap/news-french.php#item3 END:ENV END:VBKM 
Most Eureka homeowners won&#39 t collect on quake insurance The California Earthquake Authority does not expec. http bit.ly 7RY2qT
Haiti hit by largest earthquake in over 200 years AP AP The strongest earthquake in more than

In [24]:
def decontracted(phrase, tokens=True):
    """
    Replace contractions for most unambiguous cases.
    Replace apostrophe/short words.
    For example 's (a contraction of “is”)
    Ref: https://stackoverflow.com/a/47091370/10074873

    Arguments:
        phrase (str): The string to replace contractions from.
        tokens (bool): If passing tokens or whole phrases.
    
    Returns:
        phrase (str): The string without contractions.
    """
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    return phrase.strip() if tokens else phrase

### 2. Write a tokenization function to process your text data

In [29]:
def tokenize(text):
    
    # Regular expression to detect a url within text
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # get list of all urls using regex\
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # tokenize text and change all capitals to lower case
    tokens = word_tokenize(text.lower())

    # Remove stop words and replace contractions for most unambiguous cases missed from stopwords.
    stop_words = set(stopwords.words('english'))
    tokens = [decontracted(tok) for tok in tokens if tok not in stop_words]

    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Create clean tokens list iterating through each token
    clean_tokens = [lemmatizer.lemmatize(tok).strip() for tok in tokens]

    # Remove punctuation
    clean_tokens = list(filter(lambda tok: tok not in string.punctuation, clean_tokens))

    return clean_tokens

In [30]:
for message in X[15:20]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

A Comitee in Delmas 19, Rue ( street ) Janvier, Impasse Charite #2. We have about 500 people in a temporary shelter and we are in dire need of Water, Food, Medications, Tents and Clothes. Please stop by and see us.
['comitee', 'delmas', '19', 'rue', 'street', 'janvier', 'impasse', 'charite', '2.', '500', 'people', 'temporary', 'shelter', 'dire', 'need', 'water', 'food', 'medication', 'tent', 'clothes', 'please', 'stop', 'see', 'u'] 

We need food and water in Klecin 12. We are dying of hunger. Impasse Chretien Klecin 12 extended ( extension ) We are hungry and sick.
['need', 'food', 'water', 'klecin', '12.', 'dying', 'hunger', 'impasse', 'chretien', 'klecin', '12', 'extended', 'extension', 'hungry', 'sick'] 

are you going to call me or do you want me to call ou? let me know?
['going', 'call', 'want', 'call', 'ou', 'let', 'know'] 

I don't understand how to use this thing 4636.
['not', 'understand', 'use', 'thing', '4636'] 

I would like to know if the earthquake is over. Thanks
['woul

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [31]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.20, random_state=42)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [42]:
# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred, target_names=df.iloc[:, 4:].columns, zero_division=0))

                        precision    recall  f1-score   support

               related       0.84      0.96      0.89      3959
               request       0.83      0.50      0.63       902
                 offer       0.00      0.00      0.00        25
           aid_related       0.75      0.70      0.73      2156
          medical_help       0.71      0.07      0.13       431
      medical_products       0.80      0.09      0.16       264
     search_and_rescue       0.60      0.04      0.07       151
              security       0.00      0.00      0.00       106
              military       0.92      0.06      0.12       175
           child_alone       0.00      0.00      0.00         0
                 water       0.87      0.30      0.45       344
                  food       0.81      0.62      0.71       586
               shelter       0.85      0.40      0.54       487
              clothing       0.64      0.09      0.16        79
                 money       0.90      

C:\Users\Christos\miniconda3\envs\dsenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Christos\miniconda3\envs\dsenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Christos\miniconda3\envs\dsenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Christos\

In [51]:
[param for param in pipeline.get_params() if 'clf' in param and len(param) > 3]

['clf__estimator__bootstrap',
 'clf__estimator__ccp_alpha',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__max_samples',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__n_estimators',
 'clf__estimator__n_jobs',
 'clf__estimator__oob_score',
 'clf__estimator__random_state',
 'clf__estimator__verbose',
 'clf__estimator__warm_start',
 'clf__estimator',
 'clf__n_jobs']

### 6. Improve your model
Use grid search to find better parameters. 

In [54]:
# specify parameters for grid search
parameters = {
    'clf__estimator__n_estimators' : [50, 100, 200],
    'clf__estimator__min_samples_split' : [2, 3, 4]
}
# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.